In [398]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [399]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [401]:
#packege imports
from sklearn import preprocessing, linear_model
import numpy as np

import sklearn


In [403]:
#Data reading
import pandas as pd

df = pd.read_csv("new_house_data.csv")

df.drop(["id"],axis=1)
df.head()

,Unnamed: 0,id,Location,lat-lon,Area,Bed,Bath,Price
0,0,0,"Block H, Bashundhara R-A, Dhaka",0.263433,"1,600 sqft",3,3,23 Thousand
1,1,1,"Farmgate, Tejgaon, Dhaka",0.262859,900 sqft,2,2,23 Thousand
2,3,3,"Gulshan 1, Gulshan, Dhaka",0.263010,"2,200 sqft",3,4,78 Thousand
3,4,4,"Baridhara, Dhaka",0.263252,"2,200 sqft",3,3,77 Thousand
4,5,5,"Bashundhara R-A, Dhaka",0.263335,"3,000 sqft",4,5,52 Thousand


In [404]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.cluster import KMeans
import pandas as pd

# Check the column names to ensure no leading/trailing spaces
print(df.columns)
df.columns = df.columns.str.strip()  # Clean column names

# Step 1: One-Hot Encoding (if 'Location' exists)
if 'Location' in df.columns:
    encoder = OneHotEncoder()
    location_encoded = encoder.fit_transform(df[['Location']]).toarray()
    # Create a DataFrame for the one-hot encoded features
    encoded_location_df = pd.DataFrame(location_encoded, columns=encoder.get_feature_names_out(['Location']))
    # Merge the encoded columns with the main DataFrame
    df = pd.concat([df, encoded_location_df], axis=1)
    # Create a new DataFrame without the 'Location' column
    #df = df.drop(['Location'], axis=1)
    print("One-hot encoding applied.")
else:
    print("Column 'Location' does not exist for one-hot encoding.")

# Step 2: Label Encoding (if 'Location' still exists)
if 'Location' in df.columns:
    label_encoder = LabelEncoder()
    df['Location_encoded'] = label_encoder.fit_transform(df['Location'])
    # Create a new DataFrame without the 'Location' column
    df = df.drop(['Location'], axis=1)
    print("Label encoding applied.")
else:
    print("Column 'Location' does not exist for label encoding.")

# Step 3: KMeans Clustering (requires 'lat' and 'lon')
if {'lat', 'lon'}.issubset(df.columns):
    kmeans = KMeans(n_clusters=5, random_state=42)
    df['Location_cluster'] = kmeans.fit_predict(df[['lat', 'lon']])
    print("KMeans clustering applied.")
else:
    print("Columns 'lat' and 'lon' are required for KMeans clustering.")

# Debugging: Verify the updated DataFrame
print(df.head())
print(df.columns)


Index(['Unnamed: 0', 'id', 'Location', 'lat-lon', 'Area', 'Bed', 'Bath',
       'Price'],
      dtype='object')
One-hot encoding applied.
Label encoding applied.
Columns 'lat' and 'lon' are required for KMeans clustering.
   Unnamed: 0  id   lat-lon        Area  Bed  Bath        Price  \
0           0   0  0.263433  1,600 sqft    3     3  23 Thousand   
1           1   1  0.262859    900 sqft    2     2  23 Thousand   
2           3   3  0.263010  2,200 sqft    3     4  78 Thousand   
3           4   4  0.263252  2,200 sqft    3     3  77 Thousand   
4           5   5  0.263335  3,000 sqft    4     5  52 Thousand   

   Location_1st Colony, Mirpur, Dhaka  Location_Badda, Dhaka  \
0                                 0.0                    0.0   
1                                 0.0                    0.0   
2                                 0.0                    0.0   
3                                 0.0                    0.0   
4                                 0.0                  

In [405]:
# Debug: Check if 'Location' exists
print("Columns before processing:", df.columns)

# Process the "Area" column
df["Area"] = df["Area"].astype(str).map(lambda i: i.replace(" sqft", "").replace(",", ""))
df["Area"] = df["Area"].map(lambda i: int(i))  # Convert to integer after cleaning

# Process the "Price" column
def process_price(price):
    try:
        
        price = str(price)
        
        if "Thousand" in price and "." in price:
            # Replace decimal point and " Thousand"
            return float(price.replace(" Thousand", "00").replace(".", "")) 
        elif "Thousand" in price:
            # Replace " Thousand" without decimals
            return int(price.replace(" Thousand", "000"))
        if "Lakh" in price and "." in price:
            # Replace decimal point and " Thousand"
            return float(price.replace(" Lakh", "000").replace(".", "")) 
        elif "Lakh" in price:
            # Replace " Lakh" and remove any decimal points
            return int(price.replace(" Lakh", "00000"))
        else:
            # Try converting to an integer directly
            print("11111")
            return int(price)
    except (ValueError, AttributeError, TypeError) as e:
       # print(f"Error processing price: {price}. Error: {e}")
        return None  # Return None for problematic entries

# Apply the function to the Price column
df["Price"] = df["Price"].map(process_price)
print(df["Price"])

# Calculate Price per square foot
df['Price_per_sqft'] = df['Price'] / df['Area']

# Check if 'Location' exists
if 'Location' in df.columns:
    # Calculate average price per sqft by location
    location_avg_price = df.groupby('Location')['Price_per_sqft'].mean().to_dict()
    df['Location_avg_price_per_sqft'] = df['Location'].map(location_avg_price)
else:
   # print("'Location' column does not exist. Using 'Location_encoded' instead.")
    if 'Location_encoded' in df.columns:
        location_avg_price = df.groupby('Location_encoded')['Price_per_sqft'].mean().to_dict()
        df['Location_avg_price_per_sqft'] = df['Location_encoded'].map(location_avg_price)
    else:
        raise KeyError("No suitable location column found for grouping.")

# Calculate weighted price
df['Weighted_price'] = df['Price_per_sqft'] * df['Location_avg_price_per_sqft']
# print(df['Weighted_price'])
# Calculate distance from central location
from geopy.distance import geodesic

central_location = (23.8103, 90.4125)  # Example: Dhaka center
if {'lat', 'lon'}.issubset(df.columns):
    df['Distance_from_center'] = df.apply(
        lambda row: geodesic((row['lat'], row['lon']), central_location).km, axis=1
    )
else:
    print("Latitude and longitude columns ('lat', 'lon') are missing.")

# Debug: Check final columns and data
#print("Columns after processing:", df.columns)
#print(df.head())


Columns before processing: Index(['Unnamed: 0', 'id', 'lat-lon', 'Area', 'Bed', 'Bath', 'Price',
       'Location_1st Colony, Mirpur, Dhaka', 'Location_Badda, Dhaka',
       'Location_Baitul Aman Housing Society, Adabor, Dhaka',
       ...
       'Location_Taltola, Agargaon, Dhaka',
       'Location_Taltola, Khilgaon, Dhaka', 'Location_Tejgaon, Dhaka',
       'Location_Tejkunipara, Tejgaon, Dhaka',
       'Location_Uttar Badda, Badda, Dhaka', 'Location_Uttara, Dhaka',
       'Location_Wari, Sutrapur, Dhaka',
       'Location_West Kazipara, Mirpur, Dhaka',
       'Location_West Shewrapara, Mirpur, Dhaka', 'Location_encoded'],
      dtype='object', length=102)
0       23000.0
1       23000.0
2       78000.0
3       77000.0
4       52000.0
         ...   
429     22000.0
430     64000.0
431     18000.0
432     38000.0
433    145000.0
Name: Price, Length: 434, dtype: float64
Latitude and longitude columns ('lat', 'lon') are missing.


In [406]:
df.to_csv("processed_data.csv", index=False)


In [407]:
#Checking null value

df.isnull().sum()

Unnamed: 0                                 0
id                                         0
lat-lon                                    0
Area                                       0
Bed                                        0
                                          ..
Location_West Shewrapara, Mirpur, Dhaka    0
Location_encoded                           0
Price_per_sqft                             0
Location_avg_price_per_sqft                0
Weighted_price                             0
Length: 105, dtype: int64

In [408]:
#Spliting Data
#df = df.drop(["Location"],axis=1)

x = np.array(df.drop(["Price"],axis=1))
y = np.array(df["Price"])
print("x",x.shape)
print("y",y.shape)

xTrain,xTest, yTrain, yTest = sklearn.model_selection.train_test_split(x,y,
                                                                      test_size = 0.3,
                                                                      random_state = 10)
print(xTrain.shape)
print(xTest.shape)

x (434, 104)
y (434,)
(303, 104)
(131, 104)


In [409]:
#model Training

# model = linear_model.LinearRegression()
model = linear_model.LinearRegression()

model.fit(xTrain,yTrain)
accuracy = model.score(xTest,yTest)

print("Coefficiants",model.coef_)
print("Y intercept",round(model.intercept_,3))

print("Accuracy",round(accuracy*100,3),"%")

Coefficiants [-1.94995701e+00 -1.51445564e+00 -1.46894291e+08  4.66985255e+01
  1.05812589e+03 -6.26431816e+03  2.62458159e+04  3.67382184e-04
  2.26445240e+03  7.84265938e+03 -4.40944286e+04  4.42172958e+04
  1.68441852e+03 -4.20897379e+03 -3.30206953e+04  3.00544626e+04
 -2.22931291e+04  3.76614540e+04 -1.03335942e-05  2.47560288e+04
  6.34659593e+04  3.51016128e+04 -5.38753867e+04  3.54441658e+04
  2.32532819e+03  4.10910057e+04  2.55823523e+04  3.54865443e+04
  1.41239039e+04  3.52379102e+04  2.53818708e+03  3.84980857e+04
  1.32028118e+04  5.65908322e+04  3.32303885e+04 -4.53605954e+04
 -3.48228752e+04 -9.94977019e+04 -8.19110271e+04 -6.73999082e+04
  3.38433902e-04 -6.55467682e+04 -2.37359311e+04  4.12105345e+03
 -7.80848542e+04  2.16729293e+04 -5.62701670e+04 -4.27335166e-04
 -9.23525079e+04 -1.83554577e+04 -8.21417963e+04  5.98825410e+04
 -4.36201525e+04 -6.11019242e+04 -7.67126868e+04  4.99562942e-04
 -1.19099207e+04  1.06569129e+04  4.22765034e+04 -2.79429804e+04
 -4.11370370

In [410]:
#Evaluation
testvals = model.predict(xTest)
#print(testvals)

error = []

for i,testval in enumerate(testvals):
    error.append(yTest[i]-testval)
    
    print(f"Actual value {yTest[i]}; prediction {int(testval)}; Error: {round(error[i])}")
    

Actual value 64000.0; prediction 60920; Error: 3080
Actual value 68000.0; prediction 73288; Error: -5289
Actual value 11000.0; prediction 39800; Error: -28800
Actual value 245000.0; prediction 233755; Error: 11244
Actual value 32000.0; prediction 12637; Error: 19362
Actual value 12000.0; prediction 3075; Error: 8924
Actual value 115000.0; prediction 122318; Error: -7319
Actual value 22000.0; prediction 21887; Error: 112
Actual value 42000.0; prediction 46476; Error: -4477
Actual value 100000.0; prediction 112113; Error: -12114
Actual value 40000.0; prediction 52322; Error: -12323
Actual value 27000.0; prediction 49405; Error: -22405
Actual value 27000.0; prediction 14423; Error: 12576
Actual value 19000.0; prediction 16456; Error: 2544
Actual value 75000.0; prediction 91545; Error: -16545
Actual value 32000.0; prediction 29191; Error: 2809
Actual value 35000.0; prediction 38334; Error: -3334
Actual value 26000.0; prediction 28461; Error: -2461
Actual value 16000.0; prediction 18966; Er